In [21]:
import cdsapi
import os

In [22]:
DATADIR = "/scratch/snx3000/hbanderi/data/persistent_data"

c = cdsapi.Client()
MONTHS = [
    "01", "02", "03", "04", "05", "06",
    "07", "08", "09", "10", "11", "12"
]
varname = "potential_vorticity"
year = "2020"
grid = "1.0/1.0"
plevel = "850"
for month in MONTHS[:1]:
    result = c.service(
        "tool.toolbox.orchestrator.workflow",
        
        params={
            "realm": "user-apps",
            "project": 'app-c3s-daily-era5-statistics',
            "version": "master",
            "kwargs": {
                "dataset": "reanalysis-era5-pressure-levels",
                "product_type": "reanalysis",
                "variable": varname,
                "statistic": "daily_mean",
                "year": year,
                "month" : month,
                "time_zone": "UTC+00:0",
                "frequency": "1-hourly",
                "grid": plevel,
                "area": {"lat": [10, 80], "lon": [-80, 10]},
                "pressure_level": plevel,
            },
            "workflow_name": "application"
        }
    )
    c.download(result)
    
    filename = f"{DATADIR}/{varname}_{year}{month}_{grid}_{plevel}.nc"
    os.rename(result[0]["location"].split("/")[-1], filename)

2022-11-21 10:30:57,431 INFO Welcome to the CDS
2022-11-21 10:30:57,431 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/tasks/services/tool/toolbox/orchestrator/workflow/clientid-a119b45daf3641598bd7cbe52dd28a7a
2022-11-21 10:30:57,478 INFO Request is queued
2022-11-21 10:31:18,408 INFO Request is running
2022-11-21 12:51:28,435 INFO Request is failed
2022-11-21 12:51:28,435 ERROR Message: 
2022-11-21 12:51:28,436 ERROR Reason:  Traceback (most recent call last):
  File "/opt/cdstoolbox/cdscompute/cdscompute/cdshandlers/services/handler.py", line 59, in handle_request
    result = cached(context.method, proc, context, context.args, context.kwargs)
  File "/opt/cdstoolbox/cdscompute/cdscompute/caching.py", line 108, in cached
    result = proc(context, *context.args, **context.kwargs)
  File "/opt/cdstoolbox/cdscompute/cdscompute/services.py", line 124, in __call__
    return p(*args, **kwargs)
  File "/opt/cdstoolbox/cdscompute/cdscompute/services.py", line 60, in __ca

Exception: . Traceback (most recent call last):
  File "/opt/cdstoolbox/cdscompute/cdscompute/cdshandlers/services/handler.py", line 59, in handle_request
    result = cached(context.method, proc, context, context.args, context.kwargs)
  File "/opt/cdstoolbox/cdscompute/cdscompute/caching.py", line 108, in cached
    result = proc(context, *context.args, **context.kwargs)
  File "/opt/cdstoolbox/cdscompute/cdscompute/services.py", line 124, in __call__
    return p(*args, **kwargs)
  File "/opt/cdstoolbox/cdscompute/cdscompute/services.py", line 60, in __call__
    return self.proc(context, *args, **kwargs)
  File "/home/cds/cdsservices/services/retrieve.py", line 200, in execute
    **retrieve_kwargs
  File "/opt/cdstoolbox/cdscompute/cdscompute/context.py", line 301, in call
    return c.call(service, *args, **kwargs).value
  File "/opt/cdstoolbox/cdsworkflows/cdsworkflows/future.py", line 76, in value
    raise self._result
cdsworkflows.error.ClientError: None.